In [4]:
import scipy
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [5]:
L = 3 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [6]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 2 2>           26  
      1.         |2 2 1>           25  
      2.         |2 2 0>           24  
      3.         |2 1 2>           23  
      4.         |2 1 1>           22  
      5.         |2 1 0>           21  
      6.         |2 0 2>           20  
      7.         |2 0 1>           19  
      8.         |2 0 0>           18  
      9.         |1 2 2>           17  
     10.         |1 2 1>           16  
     11.         |1 2 0>           15  
     12.         |1 1 2>           14  
     13.         |1 1 1>           13  
     14.         |1 1 0>           12  
     15.         |1 0 2>           11  
     16.         |1 0 1>           10  
     17.         |1 0 0>            9  
     18.         |0 2 2>            8  
     19.         |0 2 1>            7  
     20.         |0 2 0>            6  
     21.         |0 1 2>            5  
     22.         |0 1 1>            4  
     23.   

In [7]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]]
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]]
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))


In [8]:
#basis vectors as arrays
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [9]:
#projectors on g=1 for every site
Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for state in basis_vectors:
        state = np.array(state)
        if (Gauss_law[i].expt_value(state)>=1.-1e-8) & (Gauss_law[i].expt_value(state)<=1.+1e-8):
            Proj[i] += np.outer(state,state)

In [12]:
#initialisation of the unitarily transformed Hamiltonian

linear_term = []
quadratic_term = []
interaction_p = []
interaction_m = []


for i in range(L-1):
    linear_term.append([[0.5/np.sqrt(2),i]])
    quadratic_term.append([[(-1)**i/np.sqrt(2),i,i+1]])
    interaction_p.append([
        ["+z", quadratic_term[i]],
        ["+", linear_term[i]]
    ])
    interaction_m.append([
        ["-z", quadratic_term[i]],
        ["-", linear_term[i]]
    ])

linear_term.append([[0.5/np.sqrt(2),L-1]])
interaction_p.append([
    ["+", linear_term[L-1]]
])
interaction_m.append([
    ["-", linear_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [np.zeros((basis.Ns,basis.Ns)) for i in range(L)]
h_int = np.zeros((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].toarray()@Proj[i+1]+Proj[i+1]@H_int_m[i].toarray()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.toarray()

h_int = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints


G=0
for i in range(L+1):
    G += 1e3*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.toarray()

constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term

[[ 1.7+0.j  0. +0.j  0. +0.j -0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  1. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  1.3+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0. +0.j  0. +0.j  1.4+0.j  0.5+0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0

In [101]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = np.linalg.eigh(full_ham_matrix)
print(eigenval[eigenval<=1e2])

[-1.0416023  -0.34551385 -0.34551385 -0.22065556 -0.13898669 -0.13898669
 -0.0142565   0.15174042  0.16101331  0.16101331  0.33967222  0.33967222
  0.33967222  0.33967222  0.33967222  0.35848584  0.53430948  0.53430948
  0.53479822  0.55681029  0.56101331  0.56101331  0.6         0.6
  0.6         0.63967222  0.63967222  0.69336559  0.69336559  0.72748942
  0.72748942  0.73967222  0.73967222  0.9         0.9         0.9
  0.9         0.9         0.9         0.9         0.9         0.93430948
  0.93430948  0.99336559  0.99336559  1.          1.          1.
  1.          1.          1.          1.03967222  1.03967222  1.03967222
  1.03967222  1.09336559  1.09336559  1.10404012  1.11592807  1.2
  1.2         1.2         1.2         1.2         1.2284373   1.276925
  1.276925    1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.39336559  1.39336559  1.43

In [102]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = np.linalg.eigh(constrained_full_ham_matrix)

In [103]:
#Gauge invariant part of the spectrum
print(eigenvalues[eigenvalues<=1e2])
print(eigenvectors[:,0])

[-1.0416023  -0.0142565   0.15174042  0.35848584  0.53479822  0.55681029
  1.10404012  1.11592807  1.2284373   1.48612285  1.75182104  1.76913858
  1.95519133  2.01107024  2.09579725  2.34850063  2.71262995  2.85799378
  3.6173529 ]
[ 4.11576340e-03+0.j -3.46944695e-18+0.j  2.77555756e-17+0.j
  1.53995498e-02+0.j -2.87872606e-02+0.j  2.22044605e-16+0.j
  1.11022302e-16+0.j -1.04083409e-17+0.j  5.55111512e-17+0.j
  4.33680869e-19+0.j  3.25260652e-19+0.j -3.46944695e-18+0.j
 -8.13151629e-20+0.j -4.06575815e-20+0.j  0.00000000e+00+0.j
 -2.16840434e-19+0.j -1.69406589e-21+0.j -1.69406589e-21+0.j
 -1.35525272e-20+0.j  2.96461532e-21+0.j -1.69406589e-21+0.j
  1.35525272e-20+0.j -3.38813179e-21+0.j  1.32348898e-23+0.j
 -2.11758237e-22+0.j  1.04712794e-20+0.j  2.06795153e-25+0.j
  1.53995498e-02+0.j  2.17130076e-19+0.j -4.37750665e-19+0.j
  7.30952282e-02+0.j -1.42446890e-01+0.j -4.72811760e-18+0.j
 -5.42966909e-22+0.j -1.08661880e-21+0.j  1.41211473e-21+0.j
  9.86076132e-32+0.j  0.00000000e+0

In [104]:
#initial state
string = ""
for i in range(L):
    string +="0"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.


In [105]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

8002.3


In [106]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

(2.3+0j)


In [107]:
def clause(a,b,k,m,n):
    zero = 0
    m = int(m)
    n = int(n)
    for l in range(L):
        if l == k:
            if (int(a[l])!=m) | (int(b[l])!=n):
                zero +=1
        else:
            if a[l]!=b[l]:
                zero +=1
    return zero


In [108]:
#initialisation of the equatorial rotation sigma matrices 
sigma_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]

sigma_z_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_z_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]

sigma_xy_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_xy_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]
sigma_xy_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L)]

for k in range(L):
    for i in range(basis.Ns):
        string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
        for j in range(basis.Ns):
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            
            if (clause(string_i,string_j,k,0,1)==0):
                sigma_01_temp[k][i][j] = 0.5
                sigma_xy_01_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,1,0)==0):
                sigma_01_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,1,2)==0):
                sigma_12_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,1)==0):
                sigma_12_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,2)==0):
                sigma_02_temp[k][i][j] = 0.5
            if (clause(string_i,string_j,k,2,0)==0):
                sigma_02_temp[k][i][j] = 0.5
                
            if (clause(string_i,string_j,k,0,0)==0):
                sigma_z_01_temp[k][i][j] = -1.
                sigma_z_02_temp[k][i][j] = -1.
                
            if (clause(string_i,string_j,k,1,1)==0):
                sigma_z_01_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = -1.

            if (clause(string_i,string_j,k,2,2)==0):
                sigma_z_02_temp[k][i][j] = 1.
                sigma_z_12_temp[k][i][j] = 1.


sigma_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

sigma_z_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_z_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    sigma_01 +=sigma_01_temp[i]
    sigma_12 +=sigma_12_temp[i]
    sigma_02 +=sigma_02_temp[i]
    sigma_z_01 +=sigma_z_01_temp[i]
    sigma_z_12 +=sigma_z_12_temp[i]
    sigma_z_02 +=sigma_z_02_temp[i]    


In [109]:
#initialisation of the Molmer-Sorensen gates
ms_01_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]
ms_12_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]
ms_02_temp = [np.zeros((basis.Ns,basis.Ns),dtype = complex) for i in range(L-1)]

for i in range(L-1):
    #ms_01_temp[i] = ((sigma_01_temp[i]+sigma_01_temp[i+1])@(sigma_01_temp[i]+sigma_01_temp[i+1]))/2  
    #ms_12_temp[i] = ((sigma_12_temp[i]+sigma_12_temp[i+1])@(sigma_12_temp[i]+sigma_12_temp[i+1]))/2  
    #ms_02_temp[i] = ((sigma_02_temp[i]+sigma_02_temp[i+1])@(sigma_02_temp[i]+sigma_02_temp[i+1]))/2  
    ms_01_temp[i] = ((sigma_01_temp[i]+sigma_01_temp[i+1])**2)/2  
    ms_12_temp[i] = ((sigma_12_temp[i]+sigma_12_temp[i+1])**2)/2  
    ms_02_temp[i] = ((sigma_02_temp[i]+sigma_02_temp[i+1])**2)/2  
    
ms_01 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_12 = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_02 = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L-1):
    ms_01 +=ms_01_temp[i]
    ms_12 +=ms_12_temp[i]
    ms_02 +=ms_02_temp[i]

In [110]:
def sigma(k,m,n,phi):
    sigma = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
        for j in range(0,i+1):
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            if (clause(string_i,string_j,k,m,n)==0):
                sigma[i][j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
                sigma[j][i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    return sigma
def ms(k,m,n,phi):
    return ((sigma(k,m,n,phi)+sigma(k+1,m,n,phi))**2)/2 

In [111]:
sigma_01_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_12_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_02_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)

sigma_01_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_12_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)
sigma_02_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)

ms_01_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_12_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_02_e = np.zeros((basis.Ns,basis.Ns),dtype = complex)

ms_01_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_12_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)
ms_02_o = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(1,int(L/2)):
    sigma_01_e += sigma(2*i,0,1,0)
    sigma_12_e += sigma(2*i,1,2,0)
    sigma_02_e += sigma(2*i,0,2,0)
    ms_01_e +=ms(2*i,0,1,0)
    ms_12_e +=ms(2*i,1,2,0)
    ms_02_e +=ms(2*i,0,2,0)

for i in range(int(L/2)):
    sigma_01_o += sigma(2*i+1,0,1,0)
    sigma_12_o += sigma(2*i+1,1,2,0)
    sigma_02_o += sigma(2*i+1,0,2,0)
    
for i in range(int(L/2)-1):
    ms_01_o += ms(2*i+1,0,1,0)
    ms_12_o += ms(2*i+1,1,2,0)
    ms_02_o += ms(2*i+1,0,2,0)   

In [112]:
print(np.allclose(sigma_z_12_temp[0]@sigma_01_temp[0],sigma_01_temp[0]@sigma_z_12_temp[0],atol = 1e-8,rtol=1e-8))

False


In [113]:
def rot(sigma,theta):
    return expm(-1j*theta*sigma/2)

In [114]:
def rot_sigma(k,i,j,phi,theta):
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-j][2-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csr_matrix(sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    else:
        tensprod = np.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.25*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-j][2-i] = 0.25*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csr_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,np.identity(3))+scipy.sparse.kron(np.identity(3),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    else:
        tensprod = np.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = np.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = np.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = np.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = np.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k,i,j,phi,theta)@sigma_o
    return(sigma_o)

In [125]:
phi = 0
i = 2
j = 1
sigma = np.zeros((3,3),dtype = complex)
sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
sigma[2-j][2-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
sigma = sparse.csr_matrix(sigma)
import datetime
print(datetime.datetime.now())
a = sigma*sigma
print(datetime.datetime.now())
a = sigma@sigma
print(datetime.datetime.now())
a = sigma.dot(sigma)
print(datetime.datetime.now())


2022-03-04 15:38:46.325876
2022-03-04 15:38:46.326134
2022-03-04 15:38:46.326272
2022-03-04 15:38:46.326404


In [84]:
import datetime
print(datetime.datetime.now())
a = rot_ms(0,0,1,0,np.pi/4).toarray()
print(datetime.datetime.now())
b = rot(ms_01_temp[0],np.pi/4)
print(datetime.datetime.now())
print(np.allclose(a,b,atol=1e-8,rtol=1e-8))
print(np.allclose(a@a.T.conj(),np.identity(basis.Ns),atol=1e-8,rtol=1e-8))

2022-03-04 14:01:26.532864
2022-03-04 14:01:26.541991
2022-03-04 14:01:26.554230
True
True


In [93]:
a = rot_ms_e(0,2,0,17*np.pi/18)
b = rot(ms_02_e,17*np.pi/18)
print(np.allclose(a,b,atol=1e-8,rtol=1e-8))

True


In [25]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [28]:
#cost function definition
def cost_function_sigma(theta,psi,N):
    if len(theta)!=(3*(L-1)+3*L)*N:
        print("Warning: The number of variational parameters does not match the number of layers!")
    psi_var = psi
    for i in range(N):
        for j in range(L-1):
            psi_var = rot(ms_02_temp[j],theta[(3*(L-1)+3*L)*i+6*j+5])@rot(ms_12_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j+4])@rot(ms_01_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j+3])@rot(sigma_z_02_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j+2])@rot(sigma_z_12_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j+1])@rot(sigma_z_01_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j])@psi_var
        psi_var = rot(sigma_02_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)+2])@rot(sigma_12_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)+1])@rot(sigma_01_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)])@psi_var
    expt_value = 0
    matvec = full_ham_matrix@psi_var
    for i in range(basis.Ns):
        expt_value += np.conj(psi_var[i])*matvec[i]
    cost = expt_value
    return np.real(cost)

In [29]:
#optimisation algorithm
from scipy.optimize import basinhopping
import datetime
x0 = np.zeros((3*(L-1)+3*L)*N)
resbashop = 0
print(datetime.datetime.now())
resbashop = basinhopping(cost_function_sigma, x0, minimizer_kwargs = {"args": (psi_0,N)})
print(datetime.datetime.now())

2022-02-25 19:01:56.170699


KeyboardInterrupt: 

In [ ]:
#results
print(resbashop.fun)
print(resbashop.x)


In [ ]:
#fidelity of the result
theta = resbashop.x

psi_var = psi_0
for i in range(N):
    for j in range(L-1):
        psi_var = rot(ms_02_temp[j],theta[(3*(L-1)+3*L)*i+6*j+5])@rot(ms_12_temp[j],theta[(3*(L-1)+3*L)*i+6*j+4])@rot(ms_01_temp[j],theta[(3*(L-1)+3*L)*i*i+6*j+3])@rot(sigma_02_temp[j],theta[(3*(L-1)+3*L)*i+6*j+2])@rot(sigma_12_temp[j],theta[(3*(L-1)+3*L)*i+6*j+1])@rot(sigma_01_temp[j],theta[(3*(L-1)+3*L)*i+6*j])@psi_var
psi_var = rot(sigma_02_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)+2])@rot(sigma_12_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)+1])@rot(sigma_01_temp[L-1],theta[(3*(L-1)+3*L)*i+6*(L-1)])@psi_var
print(np.abs(np.dot(np.conj(psi_var),eigenvectors[:,0]))**2)
expt_value = 0
matvec = full_ham_matrix@psi_var
for i in range(basis.Ns):
    expt_value += np.conj(psi_var[i])*matvec[i]
print(np.real(expt_value))